# Load libraries

In [22]:
# pip install fsspec / conda install -c conda-forge fsspec
# pip install s3fs
import pandas as pd
import numpy as np
import boto3
import os
import math
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# Load data from one file

In [2]:
bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment/glue-job'
file_name = 'run-1713493427814-part-r-00000'

data_location = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)

df = pd.read_csv(data_location)

print("---------------------------------")
print(f"Shape of dataframe '{df.shape}':")

---------------------------------
Shape of dataframe '(2589, 25)':


In [3]:
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df.head()

,id,fk_card_id,card_number,terminal_name,coord_x,coord_y,card_type,card_model,payment_at,mcc,amount,night_day,ic_night_day,wkend_wkday,ic_wkend_wkday,day_of_year,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_tx,time_btw_cc_tx,fraud
0,26965,843,111.111.000.843,TERM-221,222.0,261.0,CREDIT,VIRTUAL,2024-03-19 21:03:38.0,GAS,840.0,night,1,wkday,0,79.0,4,857.25,6,659.17,14,408.07,63,26,1
1,37036,636,111.111.000.636,TERM-59,38.0,34.0,CREDIT,VIRTUAL,2024-04-18 18:57:19.0,LAUNDRY,142.0,day,0,wkday,0,109.0,1,142.00,3,274.67,10,254.30,149,0,0
2,28164,538,111.111.000.538,TERM-24,77.0,45.0,CREDIT,CHIP,2024-03-23 15:47:16.0,SPORTING,408.0,day,0,wkend,1,83.0,3,209.33,3,209.33,6,207.17,8,1802,0
3,41128,517,111.111.000.517,TERM-97,8.0,85.0,CREDIT,CHIP,2024-04-30 18:52:27.0,CINEMA,106.0,day,0,wkday,0,121.0,2,109.50,2,109.50,8,210.75,22,997,0
4,37698,56,111.111.000.056,TERM-48,94.0,54.0,CREDIT,CHIP,2024-04-20 17:46:19.0,LAUNDRY,106.0,day,0,wkend,1,111.0,2,303.00,7,551.00,13,743.23,64,0,0


# Load data from a chunk of files in a folder

In [23]:
s3 = boto3.client('s3')

bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment/glue-job/'

# List objects in the specified bucket and prefix
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix_name)

# Extract the list of file keys from the response
file_keys = [obj['Key'] for obj in response.get('Contents', [])]

In [24]:
# Read each file
df = pd.DataFrame()

for obj_key in file_keys[1:]:
    # Read the object content
    obj_content = s3.get_object(Bucket=bucket_name, Key=obj_key)['Body'].read()
    print(f"Content of file '{obj_key}':")

    data_location = 's3://{}/{}'.format(bucket_name, obj_key)

    df_temp = pd.read_csv(data_location)
    df = pd.concat([df, df_temp], ignore_index=True)
    print(f"Shape of chunk '{df.shape}':")
    print("---------------------------------")

print("---------------------------------")
print(f"Shape of dataframe '{df.shape}':")


Content of file 'payment/glue-job/run-1713493427814-part-r-00000':
Shape of chunk '(2589, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00001':
Shape of chunk '(5179, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00002':
Shape of chunk '(7769, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00003':
Shape of chunk '(10359, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00004':
Shape of chunk '(12949, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00005':
Shape of chunk '(15539, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00006':
Shape of chunk '(18129, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00007':
Shape of chunk '(20

In [25]:
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df.head(3)

,id,fk_card_id,card_number,terminal_name,coord_x,coord_y,card_type,card_model,payment_at,mcc,amount,night_day,ic_night_day,wkend_wkday,ic_wkend_wkday,day_of_year,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_tx,time_btw_cc_tx,fraud
0,26965,843,111.111.000.843,TERM-221,222.0,261.0,CREDIT,VIRTUAL,2024-03-19 21:03:38.0,GAS,840.0,night,1,wkday,0,79.0,4,857.25,6,659.17,14,408.07,63,26,1
1,37036,636,111.111.000.636,TERM-59,38.0,34.0,CREDIT,VIRTUAL,2024-04-18 18:57:19.0,LAUNDRY,142.0,day,0,wkday,0,109.0,1,142.00,3,274.67,10,254.30,149,0,0
2,28164,538,111.111.000.538,TERM-24,77.0,45.0,CREDIT,CHIP,2024-03-23 15:47:16.0,SPORTING,408.0,day,0,wkend,1,83.0,3,209.33,3,209.33,6,207.17,8,1802,0


# Featuring Engeniring

In [26]:
# Filter dataframe
df = df.filter(['fraud',
                'payment_at',
                'terminal_name',
                'coord_x',
                'coord_y',
                'card_type',
                'card_model',
                'mcc',
                'amount',
                'tx_1d',
                'avg_1d',
                'tx_7d',
                'avg_7d',
                'tx_30d',
                'avg_30d',
                'time_btw_cc_tx'], axis=1)

In [27]:
df.head(3)

,fraud,payment_at,terminal_name,coord_x,coord_y,card_type,card_model,mcc,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx
0,1,2024-03-19 21:03:38.0,TERM-221,222.0,261.0,CREDIT,VIRTUAL,GAS,840.0,4,857.25,6,659.17,14,408.07,26
1,0,2024-04-18 18:57:19.0,TERM-59,38.0,34.0,CREDIT,VIRTUAL,LAUNDRY,142.0,1,142.00,3,274.67,10,254.30,0
2,0,2024-03-23 15:47:16.0,TERM-24,77.0,45.0,CREDIT,CHIP,SPORTING,408.0,3,209.33,3,209.33,6,207.17,1802


In [28]:
# Create a new column distance
df.insert(5, "distance", 0)
for ind in df.index:
    p1 = [0, 0]
    p2 = [df['coord_x'][ind], df['coord_y'][ind]]
    distance = math.sqrt(((p1[0] - p2[0]) ** 2) + ((p1[1] - p2[1]) ** 2))
    df["distance"][ind] = distance

/tmp/ipykernel_2607/3513860979.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["distance"][ind] = distance
/tmp/ipykernel_2607/3513860979.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '342.64413025761877' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df["distance"][ind] = distance


In [29]:
# extract hour
df["ts_payment_at"] = pd.to_datetime(df['payment_at'])
print(df.ts_payment_at.dtype)
df["hour"] = df.ts_payment_at.dt.hour

datetime64[ns]


In [30]:
# 2nd filter dataframe
df = df.filter(['fraud',
                'hour',
                'distance',
                'card_type',
                'card_model',
                'mcc',
                'amount',
                'tx_1d',
                'avg_1d',
                'tx_7d',
                'avg_7d',
                'tx_30d',
                'avg_30d',
                'time_btw_cc_tx'], axis=1)

In [31]:
df.head(3)

,fraud,hour,distance,card_type,card_model,mcc,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx
0,1,21,342.644130,CREDIT,VIRTUAL,GAS,840.0,4,857.25,6,659.17,14,408.07,26
1,0,18,50.990195,CREDIT,VIRTUAL,LAUNDRY,142.0,1,142.00,3,274.67,10,254.30,0
2,0,15,89.185201,CREDIT,CHIP,SPORTING,408.0,3,209.33,3,209.33,6,207.17,1802


In [32]:
df = pd.get_dummies(df, columns=['card_model', ], prefix='card_model', dtype=int, drop_first=False)
df = pd.get_dummies(df, columns=['card_type', ], prefix='card_type', dtype=int, drop_first=False)

In [33]:
df = df.filter(['fraud',
                'distance',
                'card_model_CHIP',
                'card_model_VIRTUAL',
                'card_type_CREDIT',
                'amount',
                'tx_1d',
                'avg_1d',
                'tx_7d',
                'avg_7d',
                'tx_30d',
                'avg_30d',
                'time_btw_cc_tx'], axis=1)

In [34]:
df.head(3)

,fraud,distance,card_model_CHIP,card_model_VIRTUAL,card_type_CREDIT,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx
0,1,342.644130,0,1,1,840.0,4,857.25,6,659.17,14,408.07,26
1,0,50.990195,0,1,1,142.0,1,142.00,3,274.67,10,254.30,0
2,0,89.185201,1,0,1,408.0,3,209.33,3,209.33,6,207.17,1802


# Split Data

In [35]:
train_df, validation_df, test_df= np.split(df.sample(frac=1, random_state=1729), [int(0.7 * len(df)), int(0.9 * len(df))])

print("---------------------------------")
print(f"Shape of dataframe '{train_df.shape}':")
print(f"Shape of dataframe '{validation_df.shape}':")
print(f"Shape of dataframe '{test_df.shape}':")
print("---------------------------------")

---------------------------------
Shape of dataframe '(36256, 13)':
Shape of dataframe '(10359, 13)':
Shape of dataframe '(5180, 13)':
---------------------------------


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [36]:
train_df.to_csv('train_data.csv', index=False, header=False)
validation_df.to_csv('validation_data.csv', index=False, header=False)
test_df.to_csv('test_data.csv', index=False, header=False)

In [37]:
bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name_load_file = 'payment/notebook'

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix_name_load_file, 'train/train_data.csv')).upload_file('train_data.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix_name_load_file, 'validation/validation_data.csv')).upload_file('validation_data.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix_name_load_file, 'test/test_data.csv')).upload_file('test_data.csv')